In [1]:
#import libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 
import time
import pandas as pd
import numpy as np

In [150]:
#activate chrome drivers
ser = Service("./chromedriver")
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

driver = webdriver.Chrome(options=options)
driver_links = webdriver.Chrome(options=options)
driver_2 = webdriver.Chrome(options=options)
driver_3 = webdriver.Chrome(options=options)

In [3]:
#With initial Url, get all url in each categories

driver.get('https://www.fraccion.cl/medicamentos')
url_category_list = []
category_xpath = driver.find_elements_by_xpath("// div[contains(@class, 'sub-category-item')]/div[contains(@class, 'picture')]/a")
for path in category_xpath:
    url_categories = path.get_attribute('href') # gets only the url from selenium the path
    url_category_list.append(url_categories)
print(url_category_list)

['https://www.fraccion.cl/sistema-respiratorio', 'https://www.fraccion.cl/sistema-cardiovascular', 'https://www.fraccion.cl/sistema-nervioso', 'https://www.fraccion.cl/productos-antiparasitarios-insecticidas-y-repelentes', 'https://www.fraccion.cl/dermatologicos', 'https://www.fraccion.cl/anticoncepcion', 'https://www.fraccion.cl/sistema-genitourinario-y-hormonas-sexuales', 'https://www.fraccion.cl/antiinfeccioso-de-uso-sistemico', 'https://www.fraccion.cl/gastrointestinal', 'https://www.fraccion.cl/oftalmol%C3%B3gico', 'https://www.fraccion.cl/antibi%C3%B3ticos', 'https://www.fraccion.cl/corticoides', 'https://www.fraccion.cl/anti-varicosos-y-anti-hemorroidales', 'https://www.fraccion.cl/vitaminas-y-minerales', 'https://www.fraccion.cl/huesosyrticulaciones', 'https://www.fraccion.cl/diabetes', 'https://www.fraccion.cl/dolor-y-fiebre', 'https://www.fraccion.cl/varios', 'https://www.fraccion.cl/enfermedadesespecificas', 'https://www.fraccion.cl/con-receta', 'https://www.fraccion.cl/sin-

In [4]:
#get last page number of each page in url_category_list
last_page_number_list = []
for url in url_category_list:
    driver.get(url)
    try:
        pages = driver.find_element(By.XPATH,"// div[contains(@class, 'pager')]//li[contains(@class, 'last-page')]/a")
        last_page_number = pages.get_attribute('data-pagenumber')
        last_page_number_list.append(last_page_number)
    except NoSuchElementException:  
      
        pages = driver.find_elements(By.XPATH,"// div[contains(@class, 'pager')]//li[contains(@class, 'individual-page')]/a")
        if not pages:
            last_page_number = 1
            last_page_number_list.append(last_page_number)
        
        for i in pages: 
            last_page_number = i.get_attribute('data-pagenumber')
            last_page_number_list.append(last_page_number)
        
    
    print(f'{url}: {last_page_number}') 

#change last page number to intiger for future use
last_page_number_list = [int(number) for number in last_page_number_list]

https://www.fraccion.cl/sistema-respiratorio: 7
https://www.fraccion.cl/sistema-cardiovascular: 7
https://www.fraccion.cl/sistema-nervioso: 7
https://www.fraccion.cl/productos-antiparasitarios-insecticidas-y-repelentes: 1
https://www.fraccion.cl/dermatologicos: 4
https://www.fraccion.cl/anticoncepcion: 3
https://www.fraccion.cl/sistema-genitourinario-y-hormonas-sexuales: 3
https://www.fraccion.cl/antiinfeccioso-de-uso-sistemico: 2
https://www.fraccion.cl/gastrointestinal: 11
https://www.fraccion.cl/oftalmol%C3%B3gico: 11
https://www.fraccion.cl/antibi%C3%B3ticos: 16
https://www.fraccion.cl/corticoides: 4
https://www.fraccion.cl/anti-varicosos-y-anti-hemorroidales: 1
https://www.fraccion.cl/vitaminas-y-minerales: 5
https://www.fraccion.cl/huesosyrticulaciones: 2
https://www.fraccion.cl/diabetes: 12
https://www.fraccion.cl/dolor-y-fiebre: 23
https://www.fraccion.cl/varios: 1
https://www.fraccion.cl/enfermedadesespecificas: 1
https://www.fraccion.cl/con-receta: 1
https://www.fraccion.cl/s

In [5]:
#create individual url for each categorys page
url_for_each_page_list = []

for number, category_url in zip(last_page_number_list, url_category_list):
    for p in range(number):
            url_for_each_page = category_url+f'#/pageSize=24&orderBy=10&pageNumber={p+1}'
            url_for_each_page_list.append(url_for_each_page)
        
print(len(list(url_for_each_page_list)))

107


In [160]:
url_for_each_page_list

['https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=1',
 'https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=2',
 'https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=3',
 'https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=4',
 'https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=5',
 'https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=6',
 'https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=7',
 'https://www.fraccion.cl/sistema-cardiovascular#/pageSize=24&orderBy=10&pageNumber=1',
 'https://www.fraccion.cl/sistema-cardiovascular#/pageSize=24&orderBy=10&pageNumber=2',
 'https://www.fraccion.cl/sistema-cardiovascular#/pageSize=24&orderBy=10&pageNumber=3',
 'https://www.fraccion.cl/sistema-cardiovascular#/pageSize=24&orderBy=10&pageNumber=4',
 'https://www.fraccion.cl/sistema-cardiovascul

In [174]:
#with each page link, get product url
counter = 0
all_products_list = []
for link in url_for_each_page_list:
    driver_3.get(link)
    product_xpath = driver.find_elements(By.XPATH,"// div[contains(@class, 'picture')]/a")
    counter+=1
    #print(f'{link} : {counter}')
    for path in product_xpath:
        product_url = path.get_attribute('href')
        all_products_list.append(product_url)

https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=1 : 1
https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=2 : 2
https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=3 : 3
https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=4 : 4
https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=5 : 5
https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=6 : 6
https://www.fraccion.cl/sistema-respiratorio#/pageSize=24&orderBy=10&pageNumber=7 : 7
https://www.fraccion.cl/sistema-cardiovascular#/pageSize=24&orderBy=10&pageNumber=1 : 8
https://www.fraccion.cl/sistema-cardiovascular#/pageSize=24&orderBy=10&pageNumber=2 : 9
https://www.fraccion.cl/sistema-cardiovascular#/pageSize=24&orderBy=10&pageNumber=3 : 10
https://www.fraccion.cl/sistema-cardiovascular#/pageSize=24&orderBy=10&pageNumber=4 : 11
https://www.fraccion.cl/sistema-cardiovascul

https://www.fraccion.cl/sin-receta#/pageSize=24&orderBy=10&pageNumber=1 : 94
https://www.fraccion.cl/sin-receta#/pageSize=24&orderBy=10&pageNumber=2 : 95
https://www.fraccion.cl/sin-receta#/pageSize=24&orderBy=10&pageNumber=3 : 96
https://www.fraccion.cl/salud-mujer#/pageSize=24&orderBy=10&pageNumber=1 : 97
https://www.fraccion.cl/salud-mujer#/pageSize=24&orderBy=10&pageNumber=2 : 98
https://www.fraccion.cl/salud-mujer#/pageSize=24&orderBy=10&pageNumber=3 : 99
https://www.fraccion.cl/salud-mujer#/pageSize=24&orderBy=10&pageNumber=4 : 100
https://www.fraccion.cl/salud-hombre#/pageSize=24&orderBy=10&pageNumber=1 : 101
https://www.fraccion.cl/salud-hombre#/pageSize=24&orderBy=10&pageNumber=2 : 102
https://www.fraccion.cl/salud-hombre#/pageSize=24&orderBy=10&pageNumber=3 : 103
https://www.fraccion.cl/salud-hombre#/pageSize=24&orderBy=10&pageNumber=4 : 104
https://www.fraccion.cl/salud-hombre#/pageSize=24&orderBy=10&pageNumber=5 : 105
https://www.fraccion.cl/productos#/pageSize=24&orderBy=1

In [175]:
#total number of products in the catalogue 
len(all_products_list)

2568

In [170]:
#scrape all the information from each product
list_product_variables = []

for product_url in all_products_list:
    dic_product_variables = {}

    driver_2.get(product_url) # now we are inside the individual product 
    dic_product_variables["link"] = url

    # Laboratory

    try:
        dic_product_variables["lab"] = driver_2.find_element(By.XPATH,"// div[contains(@class,'manufacturers')]//span[contains(@class,'value')]/a").text
    except NoSuchElementException:
        dic_product_variables["lab"] = 'No Laboratory'

    # Product description

    try:
        dic_product_variables["Product_description"] = driver_2.find_element(By.XPATH,"// div[contains(@class,'product-name')]/h1").text
    except NoSuchElementException:
        dic_product_variables["Product_description"] = 'No Description'

    # Normal Price

    try:  
        dic_product_variables["Price_normal"] = driver_2.find_element(By.XPATH,"// div[contains(@class,'product-price')]").text

    except NoSuchElementException:
        dic_product_variables["Price_normal"] = 'No price'

  #From the table  
    try:  
        dic_product_variables["Bioequivalent"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[1]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Bioequivalent"] = 'No info'
        
    try:  
        dic_product_variables["Recipe Type"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[2]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Recipe Type"] = 'No info'

    try:  
        dic_product_variables["Fraccionable"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[3]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Fraccionable"] = 'No info'
        
    try:  
        dic_product_variables["Cenabast"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[4]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Cenabast"] = 'No info'
        
    try:  
        dic_product_variables["DCI"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[5]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["DCI"] = 'No info'
        
    try:  
        dic_product_variables["Dosis"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[6]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Dosis"] = 'No info'

    try:  
        dic_product_variables["Measure Unit"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[7]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Measure Unit"] = 'No info'
                              
        
    try:  
        dic_product_variables["Forma Farmacéutica"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[8]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Forma Farmacéutica"] = 'No info'
        
    try:  
        dic_product_variables["Administración"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[9]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Administración"] = 'No info'
    
    try:  
        dic_product_variables["Registro ISP"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[10]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Registro ISP"] = 'No info'
                              
                              
    try:  
        dic_product_variables["CategorizacionOMS"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[11]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["CategorizacionOMS"] = 'No info'

                              
    try:  
        dic_product_variables["Grupo"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[12]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Grupo"] = 'No info'
                              
    try:  
        dic_product_variables["Laboratorio"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[13]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Laboratorio"] = 'No info'
                              
    try:  
        dic_product_variables["Referencia"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[14]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Referencia"] = 'No info'                              

    
    try:  
        dic_product_variables["RutProveedor"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[15]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["RutProveedor"] = 'No info'
                              
    try:  
        dic_product_variables["Origen"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[16]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Origen"] = 'No info' 
                              
    try:  
        dic_product_variables["Formato Venta"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[17]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Formato Venta"] = 'No info'
                              
    try:  
        dic_product_variables["Tipo Producto"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[18]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Tipo Producto"] = 'No info'
                              
    try:  
        dic_product_variables["EAN13"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[19]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["EAN13"] = 'No info'
                              
    try:  
        dic_product_variables["Valor Unidad por Caja"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[20]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Valor Unidad por Caja"] = 'No info'
                              
    try:  
        dic_product_variables["Metlife Convenio"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[21]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Metlife Convenio"] = 'No info'
                              
    try:  
        dic_product_variables["VidaCamara Convenio"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[22]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["VidaCamara Convenio"] = 'No info'
                              
    try:  
        dic_product_variables["Metlife Clase"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[23]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["Metlife Clase"] = 'No info'
                              
    try:  
        dic_product_variables["VidaCamara Clase"] = driver_2.find_element(By.XPATH,"//table[@class = 'data-table']/tbody/tr[24]/td[2]").text

    except NoSuchElementException:
        dic_product_variables["VidaCamara Clase"] = 'No info'            
        
                            

    list_product_variables.append(dic_product_variables)

df_Fraccion = pd.DataFrame(list_product_variables)
    

In [171]:
df_Fraccion

,link,lab,Product_description,Price_normal,Bioequivalent,Recipe Type,Fraccionable,Cenabast,DCI,Dosis,...,RutProveedor,Origen,Formato Venta,Tipo Producto,EAN13,Valor Unidad por Caja,Metlife Convenio,VidaCamara Convenio,Metlife Clase,VidaCamara Clase
0,https://www.fraccion.cl/productos#/pageSize=24...,PHARMA INVESTI DE CHILE S.A.,Ginkomax 80mg x 120 cápsulas,$13.990,No,Receta simple,No,No,extracto seco estandarizado de hojas de ginkgo...,80,...,,Nacional,Estuche Secundario,Marca,7803510000316,117,No,Si,,8090304
1,https://www.fraccion.cl/productos#/pageSize=24...,ITF - LABOMED FARMACÉUTICA LTDA.,Sulpilan 50mg x 30 cápsulas,$14.090,Si,Receta simple,No,No,sulpirida,50,...,,Nacional,Estuche Secundario,Marca,7809591400359,426,No,Si,,8090303
2,https://www.fraccion.cl/productos#/pageSize=24...,EUROFARMA CHILE S.A.,Esc 20mg x 30 comprimidos recubiertos,$14.690,Si,Receta simple,Si,No,escitalopram,20,...,,Nacional,Fraccionado,Marca,7891317006488,490,No,Si,,8090303
3,https://www.fraccion.cl/productos#/pageSize=24...,ASTRAZENECA S.A.,Nolvadex-D 20mg x 30 comprimidos recubiertos,$14.990,No,Receta simple,No,No,tamoxifeno,20,...,,Nacional,Estuche Secundario,Original,7896206407292,500,No,Si,,8090310
4,https://www.fraccion.cl/productos#/pageSize=24...,LABORATORIOS RAFFO S.A.,Arotex 20 mg x 30 comprimidos recubiertos,$14.990,Si,Receta simple,Si,No,paroxetina,20,...,,Nacional,Fraccionado,Marca,7501277030217,480,No,Si,,8090303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2563,https://www.fraccion.cl/productos#/pageSize=24...,LABORATORIOS ANDRÓMACO S.A.,Gamalate B6 x 30 comprimidos recubiertos,$18.990,No,Receta simple,No,No,bromo glutamato magnesio|acido gama amino buti...,75|7,...,,Nacional,Estuche Secundario,Marca,7800018128979,676,No,Si,,8090304
2564,https://www.fraccion.cl/productos#/pageSize=24...,GALENICUM HEALTH CHILE S.p.A.,Venlavitae XR 150mg x 30 cápsulas,$19.990,Si,Receta simple,No,No,venlafaxina,150,...,,Nacional,Estuche Secundario,Marca,7804653720314,1186,No,Si,,8090303
2565,https://www.fraccion.cl/productos#/pageSize=24...,NOVARTIS CHILE S.A.,Pregastar 150mg x 28 cápsulas,$19.990,Si,Receta simple,No,No,pregabalina,150,...,,Nacional,Estuche Secundario,Marca,7800065000204,682,No,Si,,8090303
2566,https://www.fraccion.cl/productos#/pageSize=24...,LABORATORIOS RECALCINE S.A.,Pregobin 75mg x 30 cápsulas,$20.790,Si,Receta simple,No,No,pregabalina,75,...,,Nacional,Estuche Secundario,Marca,7800026010945,633,No,Si,,8090303


In [172]:
df_Fraccion.shape

(2568, 28)

In [127]:
#missing values are not detected as such

pd.isnull(df_Fraccion).sum()

link                     0
lab                      0
Product_description      0
Price_normal             0
Bioequivalent            0
Recipe Type              0
Fraccionable             0
Cenabast                 0
DCI                      0
Dosis                    0
Measure Unit             0
Forma Farmacéutica       0
Administración           0
Registro ISP             0
CategorizacionOMS        0
Grupo                    0
Laboratorio              0
Referencia               0
RutProveedor             0
Origen                   0
Formato Venta            0
Tipo Producto            0
EAN13                    0
Valor Unidad por Caja    0
Metlife Convenio         0
VidaCamara Convenio      0
Metlife Clase            0
VidaCamara Clase         0
dtype: int64

In [173]:
#export to excel
df_Fraccion.to_excel ('Fraccion_catalogue.xlsx', index = False, header=True)

In [176]:
#export to csv
df_Fraccion.to_csv ('Fraccion_catalogue.csv', index = False, header=True)